**PREPARAR MATRICES PARA CONCATENAR**

`PASOS: generals`
1. Diferenciar las secuencias en el archivo donde estan alineadas.
2. Quitar identificadores.
3. Ver las diferentes. la base debe ser el fasta que este completo.
4. Añadir al final las secuencias
5. contar numero de pb
6. añadir en las secuencias por pb
7. Bioedit:
  - comprobar la longitud
  - ordenar por orden alfabetico
  - concatenar


In [1]:
# 1. ejecuta 
!pip install biopython

In [17]:
########## 2. DIFERENCIAR SECUENCIAS
# Función para analizar el identificador a partir del carácter "|"  Como las secuencias se denominan asi: >HE565550.1|Amaryllis belladonna
# mira lo que hay despues de | y si ve que se llaman igual los diferencia poniendo 1 y 2
# Función para analizar el identificador a partir del carácter "|"
def analizar_identificador(linea):
    partes = linea.split("|")
    if len(partes) > 1:
        return partes[0].strip(), partes[1].strip()
    else:
        return None, None

# Diccionario para realizar un seguimiento de cuántas veces se ha encontrado cada identificador
conteo_identificadores = {}

# Lista para almacenar las líneas del archivo modificado
lineas_modificadas = []

# Nombre del archivo de salida
nombre_archivo_salida = "C:\\Users\\arramirezj\\Desktop\\output.fas"

# Abrir el archivo FASTA
with open("C:\\Users\\arramirezj\\Desktop\\alineamiento_matk_forskaolea.fas", "r") as archivo: #####input
    for linea in archivo:
        if linea.startswith(">"):  # Comprobar si es una línea de identificador
            identificador_original, identificador = analizar_identificador(linea)
            if identificador is not None:
                if identificador not in conteo_identificadores:
                    conteo_identificadores[identificador] = 1
                else:
                    conteo_identificadores[identificador] += 1
                nuevo_identificador = f"{identificador}_{conteo_identificadores[identificador]}"
                nueva_linea = f">{identificador_original}|{nuevo_identificador}\n"
                lineas_modificadas.append(nueva_linea)
            else:
                lineas_modificadas.append(linea)  # Agregar la línea sin cambios si no se puede analizar el identificador
        else:
            lineas_modificadas.append(linea)  # Agregar la línea de secuencia sin cambios

# Escribir el archivo modificado
with open(nombre_archivo_salida, "w") as archivo_modificado:
    for linea_modificada in lineas_modificadas:
        archivo_modificado.write(linea_modificada)

print(f"Archivo modificado guardado como '{nombre_archivo_salida}'.")


Archivo modificado guardado como 'C:\Users\arramirezj\Desktop\output.fas'.


In [18]:
################### 2. QUITAR IDENTIFICADORES
# Diccionario para realizar un seguimiento de cuántas veces has encontrado cada identificador
conteo_identificadores = {}

# Lista para almacenar las líneas del archivo modificado
lineas_modificadas = []

# Nombre del archivo de salida
nombre_archivo_salida = "sinID_matk_completa-diferenciadores.fasta" ##### outuput

# Abrir el archivo FASTA
with open("output.fas", "r") as archivo: #### input
    for linea in archivo:
        if linea.startswith(">"):  # Comprobar si es una línea de identificador
            identificador = linea.strip().split("|")[-1]  # Obtener el identificador después del último "|"
            if identificador:
                if identificador not in conteo_identificadores:
                    conteo_identificadores[identificador] = 1
                else:
                    conteo_identificadores[identificador] += 1
                nuevo_identificador = f"{identificador}_{conteo_identificadores[identificador]}"
                nueva_linea = f">{nuevo_identificador}\n"
                lineas_modificadas.append(nueva_linea)
        else:
            lineas_modificadas.append(linea)  # Agregar la línea de secuencia sin cambios

# Escribir el archivo modificado
with open(nombre_archivo_salida, "w") as archivo_modificado:
    for linea_modificada in lineas_modificadas:
        archivo_modificado.write(linea_modificada)

print(f"Archivo modificado guardado como '{nombre_archivo_salida}'.")

Archivo modificado guardado como 'sinID_matk_completa-diferenciadores.fasta'.


In [ ]:
############## 3. print diferentes, base: trnl que es la mas completa, todas las secuencias deben tener el mismo n de sec
# para poder concatenar, si no quieres concatenar no hace falta este paso
def leer_fasta(archivo):
    secuencias = {}
    with open(archivo, 'r') as f:
        seq_id = None
        seq = ''
        for linea in f:
            if linea.startswith('>'):
                if seq_id:
                    secuencias[seq_id] = seq
                seq_id = linea.strip()[1:]
                seq = ''
            else:
                seq += linea.strip()
        if seq_id:
            secuencias[seq_id] = seq
    return secuencias

def comparar_secuencias(secuencias_principal, secuencias_otro):
    secuencias_faltantes = []
    for seq_id_principal in secuencias_principal:
        encontrado = False
        for seq_id_otro in secuencias_otro:
            if seq_id_principal == seq_id_otro:
                encontrado = True
                break
        if not encontrado:
            secuencias_faltantes.append(seq_id_principal)
    return secuencias_faltantes

# Leer el archivo principal con las 168 secuencias 
archivo_principal = "matK_comple.txt" #'trnL_sinID_sort_completa_lista.fas' ################ pones archivos para comparar 
secuencias_principal = leer_fasta(archivo_principal)

# Leer otro archivo FASTA con secuencias adicionales
archivo_otro = "trnL_sinID_sort_completa_lista.fas" #matK_comple.txt'
secuencias_otro = leer_fasta(archivo_otro)

# Comparar las secuencias
faltantes = comparar_secuencias(secuencias_principal.keys(), secuencias_otro.keys())

# Imprimir las secuencias faltantes en el otro archivo
print("Secuencias faltantes en", archivo_otro + ":")
for seq_id in faltantes:
    print(f">{seq_id}")
    # print(">",seq_id, end = "\n")

# Generar una variable con todos los ids de las secuencias faltantes
ids_faltantes = " ".join(faltantes)
numero = len(faltantes)
print("\nNúmero de secuencias faltantes:",numero)

############## aqui te va a salir cual id falta en que sitio y tienes que añadirlo con el script siguiente 

Secuencias faltantes en trnL_sinID_sort_completa_lista.fas:
>Gyrotaenia crassifolia_1_1

Número de secuencias faltantes: 1


In [ ]:
########### 4. copias 3 secuencias para comprobar que tienen la mismas pb, tambien lo puedes mirar en
from  Bio.Seq import Seq

my_seq1=  Seq("""------------------------------------------------------------
------------------------------AGACGCCTCCTCTTTGTATTTATTAAGACT
TTTTCTTTACGAGTATTCTA------ATTGGAATAGTCTTATTATTCCACACAA------
ATTTATAAATAAA------TCTATTACTATTTTTTCAAAAAGTAATC---------CAAG
ATTTTTCTTGTTCCTGTATAATTCTCATCTTTGTGAATACGAATCCATCTTACTTTTTCT
CCGCAACAAATCTTCTCATTTACGATTAACAGCTTCCGGTGTCTTTTTTGAGCGAATATT
TTTCTATGGAAAAATAAAGACTCCCGTAGAAGACGTCTTTGCTAATGATTTTCCGATTAG
CCTATGGTTTCTCCAGGATCTCTTCATGCATTATGTTAGATATCAAGGAAAATCTATTCT
GGCTTTAAAGGATACGGATCCGCCCCTTTTG---------ATAAATAAATGGAAATCTTT
TTTTGTCCATTTATGGCAATATAATTTTTATGTGTGGTCTCAATCAGGAAGGATGTATAT
AA------ACCAATTATGCAAGCGTTCCCTTGTCTTTTTGGGCTATCTTTCAAGTATGCG
AATAAATCTTTCAGTGGTACGGACTCAAATGCTAGAAAATTCATTTATAACGGATAAGGC
TATGAAGAAGATTGATACATTAATTCCAATTAGTCCTCTGATTGGATCGTTGGCTAAAAT
GAAATTTTGTAACGTATTAGGACATCCCGTTAGTAAGTCGACCTGGGTCGATTTATCGGA
TTTTGATATTATTGATCGATTTGTACGTATATGCAGAAAT-CTTTTTCATTATTACAGCG
GATCCTCAAAAAAAAAGAGTTTGTATCGAGTAAAATATATATTTCGACTTT---------
---""")
long1 = len(my_seq1)
print("Longi1:", long1)

my_seq2 = Seq("""------------------------------------------------------------
------------------------------AGACGCCTCCTCTTTGTATTTATTAAGACT
TTTTCTTTACGAGTATTCTA------ATTGGAATAGTCTTATTATTCCACACAA------
ATTTATAAATAAA------TCTATTACTATTTTTTCAAAAAGTAATC---------CAAG
ATTTTTCTTGTTCCTGTATAATTCTCATCTTTGTGAATACGAATCCATCTTACTTTTTCT
CCGCAACAAATCTTCTCATTTACGATTAACAGCTTCCGGTGTCTTTTTTGAGCGAATATT
TTTCTATGGAAAAATAAAGACTCCCGTAGAAGACGTCTTTGCTAATGATTTTCCGATTAG
CCTATGGTTTCTCCAGGATCTCTTCATGCATTATGTTAGATATCAAGGAAAATCTATTCT
GGCTTTAAAGGATACGGATCCGCCCCTTTTG---------ATAAATAAATGGAAATCTTT
TTTTGTCCATTTATGGCAATATAATTTTTATGTGTGGTCTCAATCAGGAAGGATGTATAT
AA------ACCAATTATGCAAGCGTTCCCTTGTCTTTTTGGGCTATCTTTCAAGTATGCG
AATAAATCTTTCAGTGGTACGGACTCAAATGCTAGAAAATTCATTTATAACGGATAAGGC
TATGAAGAAGATTGATACATTAATTCCAATTAGTCCTCTGATTGGATCGTTGGCTAAAAT
GAAATTTTGTAACGTATTAGGACATCCCGTTAGTAAGTCGACCTGGGTCGATTTATCGGA
TTTTGATATTATTGATCGATTTGTACGTATATGCAGAAAT-CTTTTTCATTATTACAGCG
GATCCTCAAAAAAAAAGAGTTTGTATCGAGTAAAATATATATTTCGACTTT---------
---""")
long2 = len(my_seq2)
print("Longi2:", long2)


my_seq3 = Seq("""---------------------ATCAGATGTTCTAATACCCTACCCGATTCATCTGGAAAT
CTTAGTTCAAACCCTTCGCTACTGGGTAAAAGACGCCTCTTTTTTGTATTTATTAGGAAT
TTTTCTTTACGAGTATTATAATTATAATTGTAATAGTCTTATTGTTC-------------
--TTCTAAATAAC------TATATTACTATTTTTTCAAAAAGTAATA---------CAAG
ATTTTTCTTGTTCCTGTATAATTCTCATCTTTGTGAATACGAATCCATCTTACTTTTTCT
ATGCAATAAATCTTCTCATTTACGATTAACAGTTTCCGGTGTCTTTTTTGAGCGAATATA
TTTCTATGGAAAAACAAAGCCTCCCGTAGAAGACGTCTTTGCTAATGATTTTTTGATTAG
CCTATGGTTTCTCCAGGATCTCTTTATGCATTATGTTAGATATCAAGGAAAATCAATTCT
TGCTTTAAAGGATACG------CCCCTTTTG---------ATAAATAAATGGAAATATTT
TTTTGTACATTTATGGCAATATAATTTTTATGTGTGGTCTCAATCAGGAAGGGTGTATAT
AA------ACCAATTATGTAAGCGTTCCCTTGTCTTTTTGGGTTATCTTTCAAGTATGCG
AATAAATCTTTCAGTTGTACGAACCCAAATGCTAGAAAATTCATTTATAACGGATAATGC
TATCAAGAAGATTGATACATTAATTCCAATTAGTCCTATGATTGCATCATTGGCTAAAAT
GAAATTTTCTAACGTATTAGGATATCCCGTTAGTAAGTTGACCTGGGTCGATTTATCGGA
TTTTGGTATTATTGATCGATTTGTGCGTATATGCAGAAAT-CTTTTTCATTATTACAGCG
GATCCTCAAAAAAAAAGAGTTTGTATCAAGTAAAATATA---------------------
---""")

long3 = len(my_seq3)
print("Longi3:", long3)


Longi1: 979
Longi2: 979
Longi3: 979


In [ ]:
# 5. Imprimir las secuencias faltantes en formato FASTA en la pantalla
# ESTO LO CPIAS AL FINAL DEL TXT O FASTA
print("Secuencias faltantes en", archivo_otro + ":")
for seq_id in faltantes:
    print(f">{seq_id}")
    seq = "?" * 979  # Secuencia ficticia de longitud 1655 caracteres
    linea_por_secuencia = 60  # Número de caracteres por línea
    for i in range(0, len(seq), linea_por_secuencia):
        print(seq[i:i+linea_por_secuencia])
    print()  # Imprimir una línea en blanco entre secuencias


Secuencias faltantes en sinID_matk_completa-diferenciadores.fasta:
>Astrothalamus reticulatus_1_1
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????????????????
????????????????????????????????????????????????

In [ ]:
#### 7. esto compruebas que tienen el mismo numero de secuencias cada arhivo:
################# CONTAJE > #############################
from google.colab import files

def contar_simbolo(archivo, simbolo):
    try:
        # Cargar el archivo desde Colab
        with open(archivo, 'r') as file:
            # Leer todas las líneas del archivo
            lineas = file.readlines()

        # Contador del símbolo
        contador_simbolo = 0

        # Iterar sobre cada línea del archivo
        for linea in lineas:
            # Contar el número de veces que aparece el símbolo en la línea
            contador_simbolo += linea.count(simbolo)

        return contador_simbolo

    except FileNotFoundError:
        return "Archivo no encontrado. Por favor, asegúrate de que el archivo esté en el directorio correcto."

# Sube tu archivo al entorno de Colab
print("Por favor, selecciona el archivo en el que quieres contar el número de símbolos:")
archivo = files.upload()

# Nombre del archivo subido (asumiendo que solo subes un archivo)
nombre_archivo = list(archivo.keys())[0]

# Contar el símbolo en el archivo
simbolo = ">"
contador = contar_simbolo(nombre_archivo, simbolo)
print(f"El número total de '{simbolo}' en el archivo '{nombre_archivo}' es: {contador}")


Por favor, selecciona el archivo en el que quieres contar el número de símbolos:


Saving matK_comple.txt to matK_comple (1).txt
El número total de '>' en el archivo 'matK_comple (1).txt' es: 169


In [ ]:
################8. Esto solo es por si trabajas en el colaboratory: descargar zip
from google.colab import files
import shutil

# Nombre de la carpeta que deseas descargar
nombre_carpeta = "/content/rbcl"

# Comprimir la carpeta en un archivo zip
shutil.make_archive("/content/rbcl", 'zip', nombre_carpeta)

# Descargar el archivo zip
files.download("/content/rbcl.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**8. Ahora con las secuencias hechas entrariamos en bioedit: ordenariamos las secuencias por orden alfabetico dandole a sort by title, comprobar la longitud, comprobar que si se han añadido mas "?" de la cuenta borrarlos (suele pasar)**